In [1]:
import numpy as np
import pandas as pd

In [2]:
label = pd.read_excel("../data/label/KETI-2017-SL-Annotation-v2_1.xlsx")
label = label[["번호", "파일명", "한국어"]]
label = label.sort_values("파일명", ascending=True)

cleared_filename = []

for idx, val in label.iterrows():
    filename = val["파일명"][:18]
    cleared_filename.append(filename)

label["파일명"] = cleared_filename
label

,번호,파일명,한국어
10460,1,KETI_SL_0000000001,0
10440,2,KETI_SL_0000000002,1
10420,3,KETI_SL_0000000003,2
10399,4,KETI_SL_0000000004,3
10379,5,KETI_SL_0000000005,4
...,...,...,...
319,10476,KETI_SL_0000010476,허리가 아파서 일어날 수 없어요
3310,10477,KETI_SL_0000010477,어떤 사람이 칼에 찔려서 피를 많이 흘리고 있어요
3899,10478,KETI_SL_0000010478,아이가 말벌에 쏘여서 기절했어요
5839,10479,KETI_SL_0000010479,무릎 인대를 다친 것 같아요


In [3]:
label2 = pd.read_excel("../data/label/KETI-2018-SL-Annotation-v1.xlsx", sheet_name='KETI-2018-수어데이터-학습용-Annotation')
label2 = label2[["번호", "파일명", "한국어"]]
label2 = label2[:33012] # label2 null값 제외
label2['번호'] = label2['번호'].astype(int) # float으로 나오는 문제 -> int 캐스팅
label2

,번호,파일명,한국어
0,10481,KETI_SL_0000010481,0
1,10482,KETI_SL_0000010482,1
2,10483,KETI_SL_0000010483,2
3,10484,KETI_SL_0000010484,3
4,10485,KETI_SL_0000010485,4
...,...,...,...
33007,43488,KETI_SL_0000043488,동생이 문틈에 손이 끼었어요
33008,43489,KETI_SL_0000043489,너무 아파요
33009,43490,KETI_SL_0000043490,빨리 와주세요
33010,43491,KETI_SL_0000043491,어떤 사람이 칼로 위협하고 있어요


In [4]:
concatenated_label = pd.concat([label, label2], axis=0)
concatenated_label

,번호,파일명,한국어
10460,1,KETI_SL_0000000001,0
10440,2,KETI_SL_0000000002,1
10420,3,KETI_SL_0000000003,2
10399,4,KETI_SL_0000000004,3
10379,5,KETI_SL_0000000005,4
...,...,...,...
33007,43488,KETI_SL_0000043488,동생이 문틈에 손이 끼었어요
33008,43489,KETI_SL_0000043489,너무 아파요
33009,43490,KETI_SL_0000043490,빨리 와주세요
33010,43491,KETI_SL_0000043491,어떤 사람이 칼로 위협하고 있어요


In [5]:
kor_string = [str(num) for num in concatenated_label['한국어']]

concatenated_label['label'] = kor_string

In [6]:
from konlpy.tag import Okt
okt = Okt()

concatenated_label['label'] = concatenated_label['label'].apply(okt.morphs)

In [7]:
from tensorflow.keras.preprocessing.text import Tokenizer

tokenizer = Tokenizer()

tokenizer.fit_on_texts(concatenated_label['label'])
tokens = tokenizer.texts_to_sequences(concatenated_label['label'])

concatenated_label['label'] = tokens

In [18]:
concatenated_label.to_csv('./concatenated_label.csv', header=True)

In [22]:
file = pd.read_csv('../test/concatenated_label.csv')
file

,Unnamed: 0,번호,파일명,한국어,label
0,10460,1,KETI_SL_0000000001,0,[167]
1,10440,2,KETI_SL_0000000002,1,[168]
2,10420,3,KETI_SL_0000000003,2,[169]
3,10399,4,KETI_SL_0000000004,3,[170]
4,10379,5,KETI_SL_0000000005,4,[171]
...,...,...,...,...,...
43487,33007,43488,KETI_SL_0000043488,동생이 문틈에 손이 끼었어요,"[84, 1, 90, 3, 103, 1, 55]"
43488,33008,43489,KETI_SL_0000043489,너무 아파요,"[500, 501]"
43489,33009,43490,KETI_SL_0000043490,빨리 와주세요,"[143, 502]"
43490,33010,43491,KETI_SL_0000043491,어떤 사람이 칼로 위협하고 있어요,"[10, 4, 1, 32, 56, 120, 23, 8]"
